# Setup Julia 

Note: When using the R package lme4, the fully maximal models often (a) fit extermely slowly, and (b) fail to converge. Following advice by Mika Braginsky (personal communication) and Douglas Bates (https://rpubs.com/dmbates/377897), we fit the models using the MixedModels package in Julia, which is faster and more likely to converge.

To allow the fit models to interface with convienant packages available in R, we used the RCall package to push fitted models to a remote R environment.

In [ ]:
# add packages if you have not done so already
    ## using Pkg
    ## Pkg.add("MixedModels")
    ## Pkg.add("RCall")
    ## Pkg.add("JellyMe4")

# load Julia packages
using MixedModels, RCall, JellyMe4;

# load R packages
R"library(emmeans)";
R"library(lme4)";
R"options(contrasts = c('contr.sum', 'contr.poly'))";

# set lmer environment to mimic lmerTest
ENV["LMER"] = "lmerTest::lmer";

## Primary analyses

In [ ]:
# open data
data = rcopy(R"readRDS('data/DF.l.inc.rds')");

# set up effect coding function
EC = EffectsCoding();

# fit model
m = 
fit(MixedModel, 
    @formula(happiness ~ trial * condition * image + (1 | ResponseId) + (1 + trial * condition * image | lab)), 
    data,
    contrasts = Dict(:trial => EC, :condition => EC, :image => EC))

In [ ]:
# save output to R
r_m = (m, data);
@rput r_m;
# R"r_m <- as_lmerModLmerTest(r_m)"

In [ ]:
Results

In [ ]:
R"summary(r_m)"

In [ ]:
# ANOVA table
R"joint_tests(r_m, 
              lmerTest.limit = 9999)"

In [ ]:
# joint tests by condition
R"joint_tests(r_m, 
              by = 'condition',
              lmerTest.limit = 9999)"

In [ ]:
# joint tests by condition and image
R"joint_tests(r_m, 
              by = c('image', 'condition'),
              lmerTest.limit = 9999)"

In [ ]:
R"emmeans(r_m, 
          pairwise ~ trial | image | condition,
          adjust = 'none')"

## Facial pose vs. filler

In [ ]:
# open data
R"data <- readRDS('data/DF.l.full.rds')";
R"data <- data[data$inc == 1 & data$inc.awa == 1, ]";
R"data <- data[data$image == "absentt", ]";
data = rcopy(R"data");

In [ ]:
R"contrasts(data$trial)"

In [ ]:
# fit model
m = 
fit(MixedModel, 
    @formula(happiness ~ trial * condition + (1 | ResponseId) + (1 + trial * condition | lab)), 
    data,
    contrasts = Dict(:trial => EC, :condition => EC))

In [ ]:
# save output to R
r_m = (m, data);
@rput r_m;
R"r_m <- as_lmerModLmerTest(r_m)"